# 3 - Item Response Theory with Stan

[![View filled on Github](https://img.shields.io/static/v1.svg?logo=github&label=Repo&message=View%20On%20Github&color=lightgrey)](https://github.com/annabavaresco/ancm2025/blob/main/docs/week_3/3_IRT_Stan.ipynb)
[![View filled in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/annabavaresco/ancm2025/blob/main/docs/week_3/3_IRT_Stan.ipynb)

In this lab, you will explore item response theory and Bayesian modelling with the Stan programming language.

## Setup

⚠️ With the following instructions, you should be able to run the assignment notebook on Colab. However, be aware that this will take significantly longer than running Stan code on your local machine. Therefore, we highly recommend running the notebook locally.

First, you need to install Stan. This may take several minutes :))

In [ ]:
import numpy as np
import pandas as pd
import stan # can be installed with pip install pystan
import json
import nest_asyncio
nest_asyncio.apply()

Next, you need to download the data and Stan template [here](https://drive.google.com/file/d/1BBeL2BtfTIBqMlJFTC_OZTUdT_pt9mpR/view?usp=share_link). Save it to your own Google Drive as in previous labs, and then mount your drive.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Unzip the files into a folder (you will be able to find this folder if you click the folder icon in your left sidebar):

In [ ]:
!unzip -qq '/content/drive/MyDrive/irt4ancm.zip'

The following cell prints a list of all of the segments used in the experiment, so that you can find and listen to the results. All of the audio was extracted from the official YouTube videos of the Eurovision Song Contest finals.

## Background

The data we'll be using in this lab comes from the Eurovision Song Contest edition of the Hooked on Music experiment. In this experiment (which you can try [here](https://app.amsterdammusiclab.nl/eurovision_2021)), people were presented with segments from Eurovision songs and had to tell whether they recognised the song or not. If their answer was 'yes', the song was muted for a few seconds and then went back on. In some trials, the song resumed at the right point. In others, it resumed a bit earlier or later. Participants were then asked whether the second segment was the 'right' continuation for the first or not. 


In [27]:
segment_df = pd.read_csv('segment_list.csv')
segment_df = segment_df.set_index('segment')
print(segment_df.shape)
segment_df.head()

(437, 7)


,song,country,year,artist,title,start_position,segment_type
segment,,,,,,,
1,1,Ukraine,2016,Jamala,1944,0.000,i
2,1,Ukraine,2016,Jamala,1944,7.925,v
3,1,Ukraine,2016,Jamala,1944,39.500,c
4,1,Ukraine,2016,Jamala,1944,72.043,v
5,1,Ukraine,2016,Jamala,1944,132.559,b


## Lab

Open the `irt4ancm.stan` file in the right-hand pane. You will make any adjstments to your model there. Here is a breakdown of what the main code blocks from `irt4ancm.stan` are doing:

```
data {
  int<lower=1> M;  // number of observations
  int<lower=1> N;  // number of participants
  int<lower=1> I;  // number of song segments
  int<lower=1> J;  // number of songs
  array[M] int<lower=0,upper=1> is_recognised;  // was the segment recognised?
  array[M] int<lower=0,upper=1> is_verified;    // was the segment verified correctly?
  array[M] int<lower=1,upper=N> participant;    // participant number
  array[M] int<lower=1,upper=I> seg;            // segment number
  array[M] int<lower=1,upper=I> song;           // segment number
  array[M] int<lower=0,upper=1> continuation_correctness;  // did the verification segment restart in the correct place?
  vector<lower=0>[M] recognition_time;            // how long did it take to recognise the segment?
  matrix[I,10] audio_features;                             // audio features for each segment
  vector[N] sophistication;                                // Goldsmith's music sophistication for each participant
}
``` 
This block is simply defining the variables corresponding to the data we're going to fit the model on.

```
parameters {
  real mu_delta;
  real<lower=0> sigma_theta;  // participant prior SD
  real<lower=0> sigma_delta;  // difficulty prior SD
  vector[N] theta;  // participant abilities
  vector[I] delta;  // segment difficulties
}
```

Here is where we declare our parameters. Any parameter that we plan to include in the model needs to be specified here. In this case, we're starting off with a Rasch (1PL) model, where the only parameters are $\theta$ (the participant's ability) and $\delta$ (the difficulty of the segment). As we experiment with more complex models (2PL, 3PL, and 4PL), we'll need to add more parameters. For each parameter, we should also specify what is our hypothesis about its distribution. This is what the following block (included inside `model`) does:

```
  // Hyperpriors
  mu_delta ~ std_normal();
  sigma_theta ~ std_normal(); // Stan automatically cuts off the negative values
  sigma_delta ~ std_normal(); // Stan automatically cuts off the negative values

  // Priors
  theta ~ normal(0, sigma_theta);
  delta ~ normal(mu_delta, sigma_delta);
```

How do we decide on which distribution to choose for each parameter? Unless you have a specific educated guess about it, starting with a normal distribution is usually a good choice. Let's now move to the core part of the code, where our model is atually defined:

```
is_verified[m] ~ bernoulli_logit(theta[participant[m]] - delta[seg[m]]);
```

This line essentially means that the variable we are trying to model (`is_verified`) is distributed as (`~`) a bernoulli logit distribution with probability $\theta-\delta$. When editing the `irt4ancm.stan` file, you might actually want to start from this line. You can choose to either predict the `is_verified` or the `is_recognised` variable. 

In [ ]:

stan_code = """
data {
  int<lower=1> M;  // number of observations
  int<lower=1> N;  // number of participants
  int<lower=1> I;  // number of song segments
  int<lower=1> J;  // number of songs
  array[M] int<lower=0,upper=1> is_recognised;  // was the segment recognised?
  array[M] int<lower=0,upper=1> is_verified;    // was the segment verified correctly?
  array[M] int<lower=1,upper=N> participant;    // participant number
  array[M] int<lower=1,upper=I> seg;            // segment number
  array[M] int<lower=1,upper=I> song;           // segment number
  array[M] int<lower=0,upper=1> continuation_correctness;  // did the verification segment restart in the correct place?
  vector<lower=0>[M] recognition_time;            // how long did it take to recognise the segment?
  matrix[I,10] audio_features;                             // audio features for each segment
  vector[N] sophistication;                                // Goldsmith's music sophistication for each participant
}

parameters {
  real mu_delta;
  real<lower=0> sigma_theta;  // participant prior SD
  real<lower=0> sigma_delta;  // difficulty prior SD
  vector[N] theta;  // participant abilities
  vector[I] delta;  // segment difficulties
}

model {
  // Hyperpriors
  mu_delta ~ std_normal();
  sigma_theta ~ std_normal(); // Stan automatically cuts off the negative values
  sigma_delta ~ std_normal(); // Stan automatically cuts off the negative values

  // Priors
  theta ~ normal(0, sigma_theta);
  delta ~ normal(mu_delta, sigma_delta);

  // Data distribution
  for (m in 1:M) {
    is_verified[m] ~ bernoulli_logit(theta[participant[m]] - delta[seg[m]]);
  }
}

"""

As we've just seen, the 'recipe' for our model is defined in the Stan file. Now we will actually compile the model. Every time you change the model, you will need to save the Stan file and recompile it by running the cell below.

💡 Tip: Colab will probably save your edits to the stan file automatically. However, be aware that the saved changes may be available only within the session. In other words, when the notebook runtime is disconnected, you may lose the edits you make to your Stan file. To make sure you don't lose your progress, you may want to use one of the following strategies:

1. Unzip the `irt4ancm.zip` folder directly in GG Drive and make sure you edit the stan file saved to your Drive

2. Alternatively, you can simply download the stan file before closing Colab. You can do that by simply hovering on the stan file on the left pane > clicking on the 3 dots > selecting 'download'.

In [ ]:
with open("all_plays.json", "rb") as my_file:
    data = json.load(my_file)

posterior = stan.build(stan_code, data=data)

14:23:05 - cmdstanpy - WARNING - CmdStanModel(model_name=...) is deprecated and will be removed in the next major version.
14:23:05 - cmdstanpy - INFO - compiling stan file /Users/anna/Documents/Code/stan/irt4ancm.stan to exe file /Users/anna/Documents/Code/stan/irt4ancm
14:23:12 - cmdstanpy - INFO - compiled model executable: /Users/anna/Documents/Code/stan/irt4ancm


We fit the model here using `all_plays.json`, which contains a complete set of data. You may find it more interesting to explore `rec_only.json` as an alternative, which contains only plays where the participant claimed to recognise the segment.

In [ ]:
fit = posterior.sample(num_chains=4, num_samples=1000)

Stan has a handy set of diagnostics that can warn you of any problems with your model fit. For the purposes of this lab, you will probably not have time to fix any problems, but you can report on them in the assignment.

If the model is (mostly) problem-free, you can look at a summary of the parameter values. Remember that we get not a specific value but rather a whole distribution on each parameter. Stan reports means, standard error and deviation, and (most popular in the literature) 5%/50%/95% quantiles.

The final three columns are convergence statistics. As a (very) rough rule of thumb, you want `N_eff` to be above 400 and `R_hat` to be less than 1.05.

In [22]:
df = fit.to_frame()
df.describe()

parameters,lp__,accept_stat__,stepsize__,treedepth__,n_leapfrog__,divergent__,energy__,mu_delta,sigma_theta,sigma_delta,...,delta.428,delta.429,delta.430,delta.431,delta.432,delta.433,delta.434,delta.435,delta.436,delta.437
count,4000.000000,4000.000000,4000.000000,4000.0,4000.0,4000.0,4000.000000,4000.000000,4000.000000,4000.000000,...,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000
mean,-5523.990000,0.849348,0.242842,4.0,15.0,0.0,5991.293050,1.256505,1.668959,0.812097,...,-0.589008,0.752237,1.492095,0.891615,1.509143,0.736791,1.221471,1.003037,1.613255,2.035318
std,25.607423,0.129524,0.006843,0.0,0.0,0.0,33.665697,0.084591,0.074378,0.044075,...,0.347382,0.388566,0.373951,0.369020,0.388765,0.366609,0.368174,0.379668,0.385189,0.403224
min,-5621.790170,0.349758,0.234787,4.0,15.0,0.0,5868.656391,0.957782,1.436793,0.661127,...,-1.863182,-0.533149,0.244123,-0.312983,0.315772,-0.629672,0.001520,-0.387035,0.509276,0.525263
25%,-5541.528995,0.763071,0.239711,4.0,15.0,0.0,5968.219056,1.200366,1.617902,0.781763,...,-0.825265,0.478205,1.227131,0.637524,1.243137,0.485602,0.968280,0.750549,1.352624,1.757781
50%,-5523.413470,0.880603,0.241429,4.0,15.0,0.0,5990.545347,1.254568,1.664574,0.810302,...,-0.590064,0.747483,1.498067,0.890994,1.499626,0.737213,1.220979,0.997684,1.611361,2.027605
75%,-5506.071780,0.957777,0.244559,4.0,15.0,0.0,6014.364416,1.313466,1.717500,0.841010,...,-0.363213,1.014039,1.751395,1.142962,1.762298,0.986630,1.467608,1.248981,1.858530,2.306078
max,-5434.705851,1.000000,0.253722,4.0,15.0,0.0,6139.062678,1.566087,1.971324,1.000106,...,0.673155,2.148030,2.728397,2.063229,2.980645,2.098693,2.588814,2.543298,2.913898,3.616050


You may want to rearrange the parameter estimates in a numpy array (or even a pandas dataframe, when your model has more parameters), so that you can look at the $\delta$ estimates for each segment.

In [30]:
deltas = fit['delta'].mean(axis=1)

In [31]:
deltas.shape

(437,)

By simply sorting the array, you can then identify the segments corresponding to the higher/lower $\delta$ values. If you'd like to listen to the actual segments, please look for the YouTube video of the target song. Due to copyright issues, we are unfortunately not allowed to provide you with downloadable `mp3` files.

In [32]:
# segments ids corresponding to lower deltas
print(np.argsort(deltas)[:10])

# segments ids corresponding to higher deltas
print(np.flip(np.argsort(deltas))[:10])

[205 427 270 271 285 254 255 192 394 269]
[261 403 117 382 353  73 120 147 143 378]


Edit `irt4ancn.stan` to try different models. Ask Ashley for help with the syntax! Handy distributions include:

*   `~ std_normal()` for a standard normal (or half-normal) distribution
*   `~ normal(mu, sigma)` for a normal distribution with specified mean and standard deviation
*   `~ lognormal(mu, sigma)` for a log-normal distribution (handy for discrimination parameters)
*   `~ bernoulli(p)` for a Bernoulli distribution parameterised by the probability of success
*   `~ bernoulli_logit(z)` for a Bernoulli distribution parameterised by the inverse logistic function of the probability of success.



The full 4PL IRT model looks like this:

$\mathrm{P}[x_{ni} = 1] = \gamma_i + (\zeta_i - \gamma_i) \frac{\mathrm{e}^{\alpha_i(\theta_n - \delta_i)}}{1 + \mathrm{e}^{\alpha_i(\theta_n - \delta_i)}}$

*   For the 3PL, $\zeta$ is fixed to 1.
*   For the 2PL, $\zeta$ is fixed to 1 and $\gamma$ is fixed to 0.
*   For the 1PL (Rasch model), $\zeta$ is fixed to 1, $\gamma$ is fixed to 0, and $\alpha$ is fixed to 1.

Don't forget to add priors as you add more parameters!

**WARNING**: In the 2PL, 3PL, and 4PL, $\theta$ needs to be distributed as a standard normal distribution and there can be no hyper-parameter $\sigma_\theta$. Otherwise, the model is not identified, and Stan will run into many problems while sampling.


### **ASSIGNMENT**

1.   Explore 1-, 2-, 3- and 4-parameter IRT models for the Hooked on Music data according to the template. Which segments are most difficult? Which are easiest? Most/least discriminating? Are the guessing parameters what you would expect? (Tip: if you're struggling with the syntax, here is the [IRT section](https://mc-stan.org/docs/stan-users-guide/regression.html#item-response-models.section) of the Stan Users Guide)
2.   Explore an alternative data model (e.g., using `rec_only.json` or focussing on `is_recognised` instead of `is_verified`), again with 1-, 2-, 3- and 4-parameter IRT models. How do your results compare to what you found in Step 1?


Write a short report (less than one page) summarising your findings and (to the extent you can) any musical explanations or surprising findings based on what you can hear in the songs.

⚠️ Please make sure you include your `irt4ancn.stan` file(s) in the assignment submission, as well as the notebook and the PDF with your report.

## Additional Resources



*   The `cmdstanpy` [documentation](https://cmdstanpy.readthedocs.io/en/v1.0.7/)
*   The Stan [User’s Guide](https://mc-stan.org/docs/stan-users-guide/])

